In [1]:
import pandas as pd
import json
import os
import glob
from pprint import pprint

In [2]:
# read the data from the json files
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
target_dir = os.path.join(parent_dir,'stock_scrape_load','data')
file_path = f"{target_dir}/*.json"

data = []

for file in glob.glob(file_path):
    with open(file, 'r') as json_file:
        json_data = json.load(json_file)
        data.append(json_data)


In [3]:
prices_df = pd.concat([pd.DataFrame(d) for d in data], ignore_index = True)


In [10]:
prices_df = prices_df.sort_values(['ticker','date'])
prices_df.head()


,ticker,date,sector,52WeekChange,ask,bid,52WeekHigh,52WeekLow,dailyOpen,previousClose,dailyVolume,quickRatio,10DayAverageVolume,marketCap
0,AAPL,2023-07-11,Technology,29.273487,188.24,188.12,194.48,124.17,189.16,188.61,46509718.0,0.764,51012280.0,2.958254e+12
100,AAPL,2023-07-12,Technology,29.273487,0.00,0.00,194.48,124.17,189.68,188.08,26845497.0,0.764,51012280.0,2.970052e+12
200,AAPL,2023-07-13,Technology,27.817070,190.33,190.32,194.48,124.17,190.50,189.77,38321242.0,0.764,52013490.0,2.996947e+12
24,ABBV,2023-07-11,Healthcare,-10.897141,135.73,135.00,168.11,130.96,134.00,134.49,5104083.0,0.659,6124920.0,2.391848e+11
124,ABBV,2023-07-12,Healthcare,-10.897141,136.54,136.47,168.11,130.96,135.82,135.57,1097186.0,0.659,6124920.0,2.399611e+11


In [11]:
ticker = pd.DataFrame(prices_df[['ticker','date','ask','bid','dailyOpen','previousClose','dailyVolume']])
ticker.head()


,ticker,date,ask,bid,dailyOpen,previousClose,dailyVolume
0,AAPL,2023-07-11,188.24,188.12,189.16,188.61,46509718.0
100,AAPL,2023-07-12,0.00,0.00,189.68,188.08,26845497.0
200,AAPL,2023-07-13,190.33,190.32,190.50,189.77,38321242.0
24,ABBV,2023-07-11,135.73,135.00,134.00,134.49,5104083.0
124,ABBV,2023-07-12,136.54,136.47,135.82,135.57,1097186.0


In [12]:
print(f"There are {len(prices_df.value_counts(['date']))} days of data represented")

There are 3 days of data represented


In [7]:
# prices_df.to_json(
#     path_or_buf = 'all_prices_xdf.json',
#     orient = 'records',
#     date_format = 'iso')


In [36]:


# for ticker, group, in prices_df.groupby(['ticker']):
    
#     day_data = []
#     for _, row in group.iterrows():
#         day_data.append({
#             'symbol': ticker,
#             'date': row['date'],
#             'dailyOpen': row['dailyOpen'],
#             'ask': row['ask'],
#             'bid': row['bid'],
#             'previousClose': row['previousClose'],
#             'dailyVolume': row['dailyVolume']
#         })
#     # tickers.append({'ticker':ticker})
#     historical_json.append({'ticker':tickers,
#                         'stats': day_data
#                         })
# with open('historical','w') as file:
#     json.dump(historical_json,file)

historical_json = []
grouped_tickers = prices_df.groupby(['ticker'])
grouped_dates = prices_df.groupby(['date'])
data = {
    "tickers": list(grouped_tickers.groups.keys()),
    "dates": list(grouped_dates.groups.keys()),
    "history": []
}

for ticker, group in grouped_tickers:
    ticker_data = {
        "ticker": ticker,
        "stats": []
    }
    
    for date, day_group in group.groupby('date'):
        date_data ={
            "date": date,
            "market" : []
        }
    
        for index, row in day_group.iterrows():
            entry_data = {
                # "symbol": row['ticker'],
                # "date": row['date'],
                "dailyOpen": row['dailyOpen'],
                "ask": row['ask'],
                "bid": row['bid'],
                "previousClose": row['previousClose'],
                "dailyVolume": row['dailyVolume']
            }
            date_data["market"].append(entry_data)
        ticker_data["stats"].append(date_data)
    data["history"].append(ticker_data)
    #     ticker_data["day"].append(day_data)
    # data["days"].append(ticker_data)

json_data = json.dumps(data, indent=4)
with open("historical.json", "w") as json_file:
    json_file.write(json_data)

In [9]:
#  THe follow is an attempt to reorganize the datafram into a structured json.  Then I realized it did not really matter
# json_data = {}

# for index, row in prices_df.iterrows():
#     date = row['date']
#     open = row['dailyOpen'], 
#     close = row['previousClose'], 
#     volume = row['dailyVolume'],
#     ticker = row['ticker'], 
#     cap = row['marketCap'], 
#     quick = row['quickRatio'],
#     sector = row['sector'],
#     low = row['52WeekLow'], 
#     high = row['52WeekHigh'], 
#     change = row['52WeekChange'],
#     ave_vol = row['10DayAverageVolume']

#     if date not in json_data:
#         json_data[date] = {'price':[],'info':[],'stats':[]}

#     json_data[date]['price'].append({'ticker':ticker,
#                                      'dailyOpen':open,
#                                      'previousClose':close,
#                                      'dailyVolume':volume
#                                                       })
#     json_data[date]['info'].append({'sector':sector,
#                                     'marketCap':cap,
#                                     })
#     json_data[date]['stats'].append({'52WeekLow':low,
#                                      '52WeekHigh':high, 
#                                      '52WeekChange': change,
#                                      '10DayAverageVolume':ave_vol
#                                      })
# # price_json = json.dumps(json_data)

# # pprint(price_json)

# prices = pd.DataFrame(json_data)
# prices.to_json('all_prices.json')

In [16]:
# Same commetn as previous cell

# output_path = f"{target_dir}/all_prices.json"

# with open('all_prices.json','w') as file:
#     file.write(price_json)